In [4]:
!pip install imgaug


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os
from glob import glob

input_image_dir = 'wb_localization_dataset\\wb_localization_dataset\\images\\train'
input_annotation_dir = 'wb_localization_dataset\\wb_localization_dataset\\labels\\train'
output_image_dir = 'output/images'
output_annotation_dir = 'output/annotations'

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_annotation_dir, exist_ok=True)

image_files = glob(os.path.join(input_image_dir, '*.jpg'))
annotation_files = glob(os.path.join(input_annotation_dir, '*.txt'))

augmentations = iaa.Sequential([
    iaa.Fliplr(0.5),  # Lật ngang
    iaa.Affine(rotate=(-20, 20)),  # Xoay ngẫu nhiên từ -20 đến 20 độ
    iaa.Multiply((0.8, 1.2)),  # Thay đổi độ sáng
    iaa.TranslateX(percent=(-0.2, 0.2)),  # Dịch chuyển theo chiều ngang
    iaa.TranslateY(percent=(-0.2, 0.2))  # Dịch chuyển theo chiều dọc
])


def read_annotations(annotation_path, image_shape):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        label = int(parts[0])
        center_x = float(parts[1]) * image_shape[1]
        center_y = float(parts[2]) * image_shape[0]
        width = float(parts[3]) * image_shape[1]
        height = float(parts[4]) * image_shape[0]
        annotations.append([label, center_x, center_y, width, height])
    return annotations


def convert_to_bboxes(annotations):
    bboxes = []
    for ann in annotations:
        label, center_x, center_y, width, height = ann
        x1 = center_x - width / 2
        y1 = center_y - height / 2
        x2 = center_x + width / 2
        y2 = center_y + height / 2
        bboxes.append(BoundingBox(x1, y1, x2, y2, label=label))
    return bboxes


for image_file in image_files:

    base_name = os.path.basename(image_file).replace('.jpg', '.txt')
    annotation_file = os.path.join(input_annotation_dir, base_name)
    
    if not os.path.exists(annotation_file):
        continue
    

    image = cv2.imread(image_file)
    annotations = read_annotations(annotation_file, image.shape)
    bboxes = convert_to_bboxes(annotations)
    

    bbs_on_image = BoundingBoxesOnImage(bboxes, shape=image.shape)
    

    for i in range(10):  
        aug_image, aug_bbs = augmentations(image=image, bounding_boxes=bbs_on_image)
        aug_image_path = os.path.join(output_image_dir, f'aug_{i}_{base_name.replace(".txt", ".jpg")}')
        aug_annotation_path = os.path.join(output_annotation_dir, f'aug_{i}_{base_name}')
        
        # Lưu ảnh tăng cường
        cv2.imwrite(aug_image_path, aug_image)
        
        # Lưu nhãn tăng cường
        with open(aug_annotation_path, 'w') as file:
            for bbox in aug_bbs.bounding_boxes:
                center_x = (bbox.x1 + bbox.x2) / 2 / image.shape[1]
                center_y = (bbox.y1 + bbox.y2) / 2 / image.shape[0]
                width = (bbox.x2 - bbox.x1) / image.shape[1]
                height = (bbox.y2 - bbox.y1) / image.shape[0]
                file.write(f"{bbox.label} {center_x} {center_y} {width} {height}\n")

print("Data augmentation for the entire folder completed successfully.")


Data augmentation for the entire folder completed successfully.


In [2]:
import cv2
import numpy as np
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os
from glob import glob
import random

# Đường dẫn đến thư mục đầu vào và đầu ra
input_image_dir = 'wb_localization_dataset\\wb_localization_dataset\\images\\train'
input_annotation_dir = 'wb_localization_dataset\\wb_localization_dataset\\labels\\train'
output_image_dir = 'output/images'
output_annotation_dir = 'output/labels'

# Tạo các thư mục đầu ra nếu chưa tồn tại
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_annotation_dir, exist_ok=True)

# Lấy danh sách tất cả các tệp ảnh và nhãn
image_files = glob(os.path.join(input_image_dir, '*.jpg'))
annotation_files = glob(os.path.join(input_annotation_dir, '*.txt'))

# Tạo các phép biến đổi
augmenters = {
    "Fliplr": iaa.Fliplr(1.0),  # Lật ngang ảnh
    "Flipud": iaa.Flipud(1.0),  # Lật dọc ảnh
    "Affine": iaa.Affine(rotate=(-20, 20)),  # Xoay ngẫu nhiên từ -20 đến 20 độ
    "Multiply": iaa.Multiply((0.8, 1.2)),  # Thay đổi độ sáng
    "TranslateX": iaa.TranslateX(percent=(-0.2, 0.2)),  # Dịch chuyển theo chiều ngang
    "TranslateY": iaa.TranslateY(percent=(-0.2, 0.2)),  # Dịch chuyển theo chiều dọc
    "Saturation": iaa.MultiplySaturation((0.5, 1.5)),  # Thay đổi mức độ bão hòa
    "Exposure": iaa.Multiply((0.7, 1.3)),  # Thay đổi độ phơi sáng
    "GaussianNoise": iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Thêm nhiễu Gaussian
    "GaussianBlur": iaa.GaussianBlur(sigma=(0, 1.0))  # Thêm Gaussian Blur
}

# Hàm để đọc nhãn từ tệp
def read_annotations(annotation_path, image_shape):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        label = int(parts[0])
        center_x = float(parts[1]) * image_shape[1]
        center_y = float(parts[2]) * image_shape[0]
        width = float(parts[3]) * image_shape[1]
        height = float(parts[4]) * image_shape[0]
        annotations.append([label, center_x, center_y, width, height])
    return annotations

# Chuyển đổi bounding boxes từ trung tâm sang góc trên trái
def convert_to_bboxes(annotations):
    bboxes = []
    for ann in annotations:
        label, center_x, center_y, width, height = ann
        x1 = center_x - width / 2
        y1 = center_y - height / 2
        x2 = center_x + width / 2
        y2 = center_y + height / 2
        bboxes.append(BoundingBox(x1, y1, x2, y2, label=label))
    return bboxes

# Lặp qua tất cả các tệp ảnh và nhãn
for image_file in image_files:
    # Tìm tệp nhãn tương ứng
    base_name = os.path.basename(image_file).replace('.jpg', '.txt')
    annotation_file = os.path.join(input_annotation_dir, base_name)
    
    if not os.path.exists(annotation_file):
        continue
    
    # Đọc ảnh và nhãn
    image = cv2.imread(image_file)
    annotations = read_annotations(annotation_file, image.shape)
    bboxes = convert_to_bboxes(annotations)
    
    # Tạo đối tượng BoundingBoxesOnImage
    bbs_on_image = BoundingBoxesOnImage(bboxes, shape=image.shape)
    
    # Chọn ngẫu nhiên một phương pháp tăng cường
    aug_name = random.choice(list(augmenters.keys()))
    augmenter = augmenters[aug_name]
    
    # Áp dụng phương pháp tăng cường đã chọn
    aug_image, aug_bbs = augmenter(image=image, bounding_boxes=bbs_on_image)
    aug_image_path = os.path.join(output_image_dir, f"{aug_name}_{base_name.replace('.txt', '.jpg')}")
    aug_annotation_path = os.path.join(output_annotation_dir, f"{aug_name}_{base_name}")
    
    # Lưu ảnh tăng cường
    cv2.imwrite(aug_image_path, aug_image)
    
    # Lưu nhãn tăng cường
    with open(aug_annotation_path, 'w') as file:
        for bbox in aug_bbs.bounding_boxes:
            center_x = (bbox.x1 + bbox.x2) / 2 / image.shape[1]
            center_y = (bbox.y1 + bbox.y2) / 2 / image.shape[0]
            width = (bbox.x2 - bbox.x1) / image.shape[1]
            height = (bbox.y2 - bbox.y1) / image.shape[0]
            file.write(f"{bbox.label} {center_x} {center_y} {width} {height}\n")
    
    # Áp dụng kết hợp tất cả các phương pháp
    combined_augmenter = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(rotate=(-20, 20)),
        iaa.Multiply((0.8, 1.2)),
        iaa.TranslateX(percent=(-0.2, 0.2)),
        iaa.TranslateY(percent=(-0.2, 0.2)),
        iaa.MultiplySaturation((0.5, 1.5)),
        iaa.Multiply((0.7, 1.3)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),
        iaa.GaussianBlur(sigma=(0, 1.0))
    ])
    aug_image_combined, aug_bbs_combined = combined_augmenter(image=image, bounding_boxes=bbs_on_image)
    aug_image_combined_path = os.path.join(output_image_dir, f"Combined_{base_name.replace('.txt', '.jpg')}")
    aug_annotation_combined_path = os.path.join(output_annotation_dir, f"Combined_{base_name}")
    
    # Lưu ảnh tăng cường kết hợp
    cv2.imwrite(aug_image_combined_path, aug_image_combined)
    
    # Lưu nhãn tăng cường kết hợp
    with open(aug_annotation_combined_path, 'w') as file:
        for bbox in aug_bbs_combined.bounding_boxes:
            center_x = (bbox.x1 + bbox.x2) / 2 / image.shape[1]
            center_y = (bbox.y1 + bbox.y2) / 2 / image.shape[0]
            width = (bbox.x2 - bbox.x1) / image.shape[1]
            height = (bbox.y2 - bbox.y1) / image.shape[0]
            file.write(f"{bbox.label} {center_x} {center_y} {width} {height}\n")

print("Data augmentation for the entire folder completed successfully.")


Data augmentation for the entire folder completed successfully.
